In [2]:
!pip install tqdm boto3 requests regex sentencepiece sacremoses datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 108.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This

## [MY CODE] Set Tokenizer
- Hugging Face의 Transformer 아키텍처 기반의 Distilbert 모델 사용
- distilbert는 knowledge distillation 기법을 사용하여 Bert 모델 대비 성능은 유지하면서 속도와 크기를 줄인 경량화 모델
- knowledge distillation 기법은 Teacher model / Student model을 이용하여 훈련시키는 기법
- uncased 모델은 텍스트 입력에서 대소문자 구분은 수행하지 않음

In [3]:
import torch
from transformers import AutoTokenizer
from datasets import load_dataset
from torch.utils.data import DataLoader

# tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'distilbert-base-uncased')
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

## [MY CODE] Load Dataset
- Huggingface dataset의 fancyzhx/ag_news을 사용
- trainset volume: 120,000
- testset volume: 7,600

In [4]:
ds = load_dataset("fancyzhx/ag_news")
ds

README.md:   0%|          | 0.00/8.07k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7600
    })
})

## [MY CODE] define collate_fn
- collate_fn은 pytorch dataloader에서 사용하는 함수로, 배치 데이터를 전처리하여 모델에 입력할 수 있는 형태로 변환하는 역할을 수행
- truncation의 목적은,


## --- Question
1. text는 tokenizer를 거치는데, 왜 label은 tokenizer를 거치지 않나요?
2. truncation 관련 부분들이 어디까지인가요? (max_len, padding, ...)

In [5]:
def collate_fn(batch):
  # 토큰화된 텍스트의 최대 길이를 400으로 제한할 수 있도록 변수 값 설정. 초과되면 자르고(truncation), 부족하면 패딩할 수 있음(padding)
  max_len = 400
  texts, labels = [], []

  # 준비된 데이터셋의 features: ['text', 'label'] 이므로 이에 맞게 분류
  for row in batch:
    labels.append(row['label'])
    texts.append(row['text'])

  # 데이터셋에서 추출한 text, label 특성 별 tensor 형태로 변환
  """
  torch.LongTensor(): 입력 값이나 토큰화된 결과를 PyTorch의 텐서로 변환

  tokenizer(): 텍스트를 토큰화하는 함수
  - padding=True: 배치 내 가장 긴 시퀀스에 맞춰 작업, 남는 길이는 패딩 처리
  - truncation=True: 최대 길이를 초과하면 자르도록 수행 (-> 과제에서 관련 부분 지우라고 했는데, 확인 필요)
  - max_length=max_len: 시퀀스의 최대 길이를 max_len값(예: 400) 만큼 제한
  - .input_ids: 토크나이저가 반환한 입력 ID(=숫자로 변환된 텍스트)를 가져옴
    - (참고) 텍스트가 토큰화 된 결과 예시: [input_ids, attention_mask, ...]
  """
  texts = torch.LongTensor(tokenizer(texts, padding=True, truncation=True, max_length=max_len).input_ids)
  labels = torch.LongTensor(labels)

  return texts, labels

# [Not My Code] define dataloader
- pytorch의 data loader는 데이터셋을 배치 단위로 나누고 배치를 반복 가능한 형태로 제공
- trainset과 testset을 구분하여 data loader를 정의
- batch size (정수값): 데이터셋의 배치 사이즈를 어느 크기로 나눌지 설정
- shuffle (True/False): 데이터셋 샘플링 순서의 무작위 여부 설정
- collate_fn : 사용자 정의 전처리 방법 입력

In [6]:
train_loader = DataLoader(
    ds['train'], batch_size=64, shuffle=True, collate_fn=collate_fn
)

test_loader = DataLoader(
    ds['test'], batch_size=64, shuffle=False, collate_fn=collate_fn
)

## [MY CODE] load Model
- 실제 분류를 수행할 Distilbert 모델 로드
- 특히, classification을 수행할 모델을 로드(label은 dataset의 classes 참고)
  - AutoModel.from_pretrained()로 로드하면 bert모델의 output이 마지막 레이어의 히든 스테이트를 반환
  - 따라서, 최종적으로 분류 작업을 수행하려면 추가적인 **헤드(head)**를 수동으로 추가해야 함.
- 예시:

```python
# AutoModel

from transformers import AutoModel

model = AutoModel.from_pretrained("distilbert-base-uncased")
outputs = model(input_ids)  # 히든 스테이트 반환
hidden_states = outputs.last_hidden_state

# AutoModelForSequenceClassification

from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=4
)
outputs = model(input_ids)  # 로짓 반환
logits = outputs.logits
predicted_class = logits.argmax(dim=-1)
```

In [10]:
from transformers import AutoModel, AutoModelForSequenceClassification

# model = torch.hub.load('huggingface/pytorch-transformers', 'model', 'distilbert-base-uncased')
# model = AutoModel.from_pretrained("distilbert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=4) # World, Sports, Business, Sci/Tech
model

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


출력 결과를 통해 우리는 DistilBERT의 architecture는 일반적인 Transformer와 동일한 것을 알 수 있습니다.
Embedding layer로 시작해서 여러 layer의 Attention, FFN를 거칩니다.

이제 DistilBERT를 거치고 난 `[CLS]` token의 representation을 가지고 text 분류를 하는 모델을 구현합시다.

In [9]:
from torch import nn

class TextClassifier(nn.Module):
  def __init__(self):
    super().__init__()

    # self.encoder = torch.hub.load('huggingface/pytorch-transformers', 'model', 'distilbert-base-uncased')
    self.encoder = AutoModel.from_pretrained("distilbert-base-uncased")
    self.classifier = nn.Linear(768, 1)

  def forward(self, x):
    x = self.encoder(x)['last_hidden_state']
    x = self.classifier(x[:, 0])

    return x

model = TextClassifier() # >>?>??????? 이러면 model 덮어써지는거 아닌가

위와 같이 `TextClassifier`의 `encoder`를 불러온 DistilBERT, 그리고 `classifier`를 linear layer로 설정합니다.
그리고 `forward` 함수에서 순차적으로 사용하여 예측 결과를 반환합니다.

다음은 마지막 classifier layer를 제외한 나머지 부분을 freeze하는 코드를 구현합니다.

In [ ]:
for param in model.encoder.parameters():
  param.requires_grad = False

위의 코드는 `encoder`에 해당하는 parameter들의 `requires_grad`를 `False`로 설정하는 모습입니다.
`requires_grad`를 `False`로 두는 경우, gradient 계산 및 업데이트가 이루어지지 않아 결과적으로 학습이 되지 않습니다.
즉, 마지막 `classifier`에 해당하는 linear layer만 학습이 이루어집니다.
이런 식으로 특정 부분들을 freeze하게 되면 효율적으로 학습을 할 수 있습니다.

마지막으로 이전과 같은 코드를 사용하여 학습 결과를 확인해봅시다.

In [ ]:
from torch.optim import Adam
import numpy as np
import matplotlib.pyplot as plt


lr = 0.001
model = model.to('cuda')
loss_fn = nn.BCEWithLogitsLoss()

optimizer = Adam(model.parameters(), lr=lr)
n_epochs = 10

for epoch in range(n_epochs):
  total_loss = 0.
  model.train()
  for data in train_loader:
    model.zero_grad()
    inputs, labels = data
    inputs, labels = inputs.to('cuda'), labels.to('cuda').float()

    preds = model(inputs)[..., 0]
    loss = loss_fn(preds, labels)
    loss.backward()
    optimizer.step()

    total_loss += loss.item()

  print(f"Epoch {epoch:3d} | Train Loss: {total_loss}")

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


Epoch   0 | Train Loss: 234.78025314211845
Epoch   1 | Train Loss: 199.59758660197258
Epoch   2 | Train Loss: 186.83845001459122
Epoch   3 | Train Loss: 179.1590757369995
Epoch   4 | Train Loss: 174.46909701824188
Epoch   5 | Train Loss: 171.1713205575943
Epoch   6 | Train Loss: 169.39869466423988
Epoch   7 | Train Loss: 167.80235224962234
Epoch   8 | Train Loss: 165.50148677825928
Epoch   9 | Train Loss: 164.10345768928528


In [ ]:
def accuracy(model, dataloader):
  cnt = 0
  acc = 0

  for data in dataloader:
    inputs, labels = data
    inputs, labels = inputs.to('cuda'), labels.to('cuda')

    preds = model(inputs)
    # preds = torch.argmax(preds, dim=-1)
    preds = (preds > 0).long()[..., 0]

    cnt += labels.shape[0]
    acc += (labels == preds).sum().item()

  return acc / cnt


with torch.no_grad():
  model.eval()
  train_acc = accuracy(model, train_loader)
  test_acc = accuracy(model, test_loader)
  print(f"=========> Train acc: {train_acc:.3f} | Test acc: {test_acc:.3f}")

=========> Train acc: 0.837 | Test acc: 0.828


Loss가 잘 떨어지고, 이전에 우리가 구현한 Transformer보다 더 빨리 수렴하는 것을 알 수 있습니다.